In [14]:
from collections import Counter
import random
import string
import Levenshtein

In [84]:
PRINT_LEN = 50

In [141]:
def get_accuracy(pred, label):
    corrects = 0
    for i in range(len(pred)):
        if pred[i] == label[i]:
            corrects += 1
    acc = corrects / len(pred)
    distance = Levenshtein.distance(pred, label)
    print(f'Accuracy: {acc:.3f}  Levenstain distance: {distance}')
    return acc, distance

In [142]:
data_path = 'texts/WarAndPeace.txt'
with open(data_path, 'r') as f:
    corpus = f.read()
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
corpus = corpus.lower().replace('\n', ' ').replace('\t', ' ')
corpus = ''.join([x for x in corpus if x in alphabet])
corpus = ' '.join(corpus.split())
corpus[:PRINT_LEN]


'война и мир самый известный роман льва николаевича'

### 1. Базовый частотный метод

In [143]:
unigrams = Counter(corpus).most_common()
unigrams[:5]

[(' ', 103408), ('о', 61282), ('а', 45209), ('е', 42519), ('и', 35838)]

In [144]:
test = corpus[:10000]

unique_chars = list(set(test))
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))
trans_table = str.maketrans(encoder_dict)

test_encoded = test.translate(trans_table)
encoded_unigrams = Counter(test_encoded).most_common()
print('true:   ', test[:PRINT_LEN])
print('encoded:', test_encoded[:PRINT_LEN])

true:    война и мир самый известный роман льва николаевича
encoded: яжквезьзньтзюенакзьыяоюэвакзтжневзфъяезвьшжфеояьхе


In [145]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
for i in range(40):
    decoder_dict = dict(zip([x[0] for x in encoded_unigrams[:i + 1]], [x[0] for x in unigrams[:i + 1]]))
    trans_table = str.maketrans(decoder_dict)
    test_decoded = test_encoded.translate(trans_table)
    print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
    get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  яжквезьзньтзюенакзьыяоюэвакзтжневзфъяезвьшжфеояьхе
top  1 : яжкве ь ньт юенак ьыяоюэвак тжнев фъяе вьшжфеояьхе   Accuracy: 0.162  Levenstain distance: 8351
top  2 : яокве ь ньт юенак ьыяоюэвак тонев фъяе вьшофеояьхе   Accuracy: 0.251  Levenstain distance: 7488
top  3 : яокве ь ньт юенак ьыяаюэвак тонев фъяе вьшофеаяьхе   Accuracy: 0.251  Levenstain distance: 7479
top  4 : яокве ь ньт юенак ьыяаюэвак тонев фъяе вьшофеаяьхе   Accuracy: 0.251  Levenstain distance: 7479
top  5 : яокие ь ньт юенак ьыяаюэиак тонеи фъяе иьшофеаяьхе   Accuracy: 0.251  Levenstain distance: 7470
top  6 : яокие н ннт юенак ныяаюэиак тонеи фъяе иншофеаянхе   Accuracy: 0.251  Levenstain distance: 7469
top  7 : яокие н ннт юенак ныяаютиак тонеи фъяе иншофеаянхе   Accuracy: 0.301  Levenstain distance: 6976
top  8 : сокие н ннт юенак нысаютиак тонеи фъсе иншофеаснхе   Accuracy: 0.301  Levenstain distance: 6976
top  9 : сокие н ннт ленак нысалтиак т

In [146]:
decoder_dict

{'з': ' ',
 'ж': 'о',
 'о': 'а',
 'е': 'е',
 'в': 'и',
 'ь': 'н',
 'э': 'т',
 'я': 'с',
 'ю': 'л',
 'т': 'в',
 'ф': 'р',
 'ш': 'к',
 'н': 'д',
 ' ': 'м',
 'п': 'у',
 'г': 'п',
 'л': 'я',
 'ы': 'г',
 'а': 'ь',
 'ъ': 'ы',
 'х': 'з',
 'у': 'б',
 'р': 'ч',
 'к': 'й',
 'ё': 'ж',
 'б': 'ш',
 'ц': 'х',
 'м': 'ю',
 'с': 'ц',
 'д': 'э',
 'щ': 'щ',
 'и': 'ф',
 'ч': 'ё',
 'й': 'ъ'}

### 2. Биграммный частотный метод

In [266]:
bigrams = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common()
print(f'bigrams: {len(bigrams)}')
bigrams[:5]

bigrams: 796


[('о ', 13316), ('и ', 11398), ('а ', 10596), ('е ', 10040), (' с', 9863)]

In [148]:
unique_chars = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common() 
unique_chars = [x[0] for x in unique_chars]
unique_chars_shuffled = unique_chars.copy()
random.Random(42).shuffle(unique_chars_shuffled)

encoder_dict = dict(zip(unique_chars, unique_chars_shuffled))

test_encoded = ''.join([encoder_dict[test[i:i+2]] for i in range(0, len(test), 2)])
encoded_bigrams = Counter(test_encoded[x:x+2] for x in range(len(test_encoded) - 1)).most_common() 
print(test[:100])
print(test_encoded[:100])

война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя
вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеуёмршйшё цесржбвзсрйллбяладзцдшядсебакдвзлбнэнбстяг


In [149]:
print('true:    ', test[:PRINT_LEN])
print('encoded: ', test_encoded[:PRINT_LEN])
for i in range(20):
    decoder_dict = dict(zip([x[0] for x in encoded_bigrams[:i + 1]], [x[0] for x in bigrams[:i + 1]]))
    test_decoded = ''.join([decoder_dict[test_encoded[i:i+2]] if test_encoded[i:i+2] in decoder_dict else test_encoded[i:i+2] for i in range(0, len(test_encoded), 2)])
    print(f'top {i + 1:2} :', test_decoded[:PRINT_LEN], end='   ')
    get_accuracy(test, test_decoded);

true:     война и мир самый известный роман льва николаевича
encoded:  вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу
top  1 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.047  Levenstain distance: 8815
top  2 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.057  Levenstain distance: 8761
top  3 : вчькдмо тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.066  Levenstain distance: 8727
top  4 : вчьке о тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.076  Levenstain distance: 8676
top  5 : вчьке о тоьснбуепгсебайшктпгядъессгшруепмзршюсэдеу   Accuracy: 0.084  Levenstain distance: 8655
top  6 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8631
top  7 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8646
top  8 : вчьке о тоьснбуепгсебайшктпгядъессгшру пмзршюсэдеу   Accuracy: 0.092  Levenstain distance: 8621
top  9 : вчьке о тоьснбуепгсебайшктпгя

In [150]:
decoder_dict

{'рз': 'о ',
 'лб': 'и ',
 'це': 'а ',
 'дм': 'е ',
 'ео': ' с',
 'еп': ' п',
 ' о': ' в',
 'ев': ' н',
 'дш': 'то',
 'ач': ' о',
 'т ': 'я ',
 'вш': ' к',
 'зж': ' и',
 'ые': 'ст',
 'шр': 'ь ',
 'пг': 'на',
 'йш': 'го',
 'шв': 'ал',
 'сш': 'не',
 'дк': 'по'}

### 3. MCMC-сэмплированиe

In [151]:
import numpy as np

In [348]:
bigrams = Counter(corpus[x:x+2] for x in range(len(corpus) - 1)).most_common()
bigrams = [x[0] for x in bigrams]
bigrams[:5], len(bigrams)

(['о ', 'и ', 'а ', 'е ', ' с'], 796)

In [349]:
token2id = {token: i for i, token in enumerate(bigrams)}
id2token = {i: token for token, i in token2id.items()}
token2id;

static_distr = np.zeros((len(bigrams), len(bigrams)))
static_distr.shape

(796, 796)

In [350]:
for i in range(len(corpus) - 2):
    cur_token = token2id[corpus[i:i+2]]
    next_token = token2id[corpus[i+1:i+3]]
    static_distr[cur_token][next_token] += 1

sort_keys = static_distr.max(axis=1).argsort()[::-1]
static_distr_sorted = np.array(list(map(lambda x: x[sort_keys], static_distr)))[sort_keys]
bigrams_sorted = [x for _, x in sorted(zip(sort_keys, bigrams), reverse=True)]
(static_distr_sorted != 0).sum()

7145

In [342]:
# static_distr = np.array([[4, 5, 6], [7, 8, 9], [1, 2, 3]])
# sort_keys = static_distr.max(axis=1).argsort()[::-1]
# static_distr_sorted = np.array(list(map(lambda x: x[sort_keys], static_distr)))[sort_keys]
# static_distr_sorted, sort_keys

(array([[8, 7, 9],
        [5, 4, 6],
        [2, 1, 3]]),
 array([1, 0, 2]))

In [344]:
bigrams[:5], bigrams_sorted[:5]

(['о ', 'и ', 'а ', 'е ', ' с'], ['км', 'йй', 'тё', ' ъ', 'ъ '])

In [353]:
static_distr_sorted[0].argmax()

44

In [354]:
bigrams_sorted[0]

'км'

In [355]:
bigrams_sorted[44]

'пц'

In [321]:

static_distr_sorted.diagonal().argmax()

373

In [310]:
static_distr_sorted[373, 373], bigrams[373]

(7.0, 'ип')

In [311]:
'ипип' in corpus

False

In [283]:
true_distr_id = {}
true_distr_token = {}
for i, id in enumerate(static_distr_sorted.argmax(axis=1)):
    true_distr_id[i] = id
    true_distr_token[id2token[i]] = id2token[id]

In [284]:
test_encoded

'вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеуёмршйшё цесржбвзсрйллбяладзцдшядсебакдвзлбнэнбстягт шшжаюсифнкпьащипутцетояднр гзжбх оуябъсаппрблюфкдавзмм ощнсшыснгдмбакж мдшяшшаипояцетидммпцеш зйт йрсевз овшя ибрзцеткуврзрзжмйшрзцерыашйшба озкр ымющлбцепэяфлбрзвгвсшллбцевчьклбрзтокедшр лчпгачвбяидхшашнвшеодожчвбмпрдйшвчвбяд ярбеозддмвгсршвктеошвовлгтмдизжбхафтуюхдпиошнмклмрзрзкдл тоеовшзйушдпсаачлчъ идвмюшйшвчожубршйлаяжуъ ащшшшлрзипфмойдблчъ бажмсшеовшящтилбжкмгьбкт тдур р убящсдиукронба одшг р уонилбсгиувзйлнщюнбрёмршйшмлеуйшыеьдэхалгчзыл т сшдкеуйшыешшг рэывюнепмзршюсэдшиачкиачпгвчькдмрзтоьсачвбеуйшыеьдэхалжэдшшам л дккрпгбаищпыдпрцепипояцебунцыеянгтдмрзцкббдмйшящрзпиубршоблбсржбвгя йшрэтоуябзбърбубти жыжстепг осеозыдшкеюхпчтурмсфм ьбеосдечлбпившсрашстгчачожепзыеомлевжэпэрзлчжэрддштдвчьбствш члбшврпзчл еойцзк йсаеяеойцожжжддлюачб  тзттцыдйшдмпчяфцедкбачегчачт сстлеягыимифдкрурв цгшугеплбяилоеосду еплбяладафмлеосду еплбкрпгбаищпыдпрцдадхеосдб вчыдстепипмшсшш шеохстс яляфйшхеазлбд

In [201]:
encoded_bigrams = Counter(test_encoded[x:x+2] for x in range(len(test_encoded) - 1)).most_common() 
encoded_bigrams = [x[0] for x in encoded_bigrams]
len(encoded_bigrams)

961

In [211]:
'чь' in encoded_bigrams

True

In [207]:
encoded_token2id = {token: i for i, token in enumerate(encoded_bigrams)}
encoded_id2token = {i: token for token, i in encoded_token2id.items()}

In [208]:
encoded_static_distr = np.zeros((len(encoded_bigrams), len(encoded_bigrams)))
encoded_static_distr.shape

(961, 961)

In [210]:
test_encoded

'вчькдмрзтоьснбуепгсебайшктпгядъессгшруепмзршюсэдеуёмршйшё цесржбвзсрйллбяладзцдшядсебакдвзлбнэнбстягт шшжаюсифнкпьащипутцетояднр гзжбх оуябъсаппрблюфкдавзмм ощнсшыснгдмбакж мдшяшшаипояцетидммпцеш зйт йрсевз овшя ибрзцеткуврзрзжмйшрзцерыашйшба озкр ымющлбцепэяфлбрзвгвсшллбцевчьклбрзтокедшр лчпгачвбяидхшашнвшеодожчвбмпрдйшвчвбяд ярбеозддмвгсршвктеошвовлгтмдизжбхафтуюхдпиошнмклмрзрзкдл тоеовшзйушдпсаачлчъ идвмюшйшвчожубршйлаяжуъ ащшшшлрзипфмойдблчъ бажмсшеовшящтилбжкмгьбкт тдур р убящсдиукронба одшг р уонилбсгиувзйлнщюнбрёмршйшмлеуйшыеьдэхалгчзыл т сшдкеуйшыешшг рэывюнепмзршюсэдшиачкиачпгвчькдмрзтоьсачвбеуйшыеьдэхалжэдшшам л дккрпгбаищпыдпрцепипояцебунцыеянгтдмрзцкббдмйшящрзпиубршоблбсржбвгя йшрэтоуябзбърбубти жыжстепг осеозыдшкеюхпчтурмсфм ьбеосдечлбпившсрашстгчачожепзыеомлевжэпэрзлчжэрддштдвчьбствш члбшврпзчл еойцзк йсаеяеойцожжжддлюачб  тзттцыдйшдмпчяфцедкбачегчачт сстлеягыимифдкрурв цгшугеплбяилоеосду еплбяладафмлеосду еплбкрпгбаищпыдпрцдадхеосдб вчыдстепипмшсшш шеохстс яляфйшхеазлбд

In [213]:
for i in range(len(test_encoded) - 2):
    cur_token = encoded_token2id[test_encoded[i:i+2]]
    next_token = encoded_token2id[test_encoded[i+1:i+3]]
    encoded_static_distr[cur_token][next_token] += 1

In [215]:
(encoded_static_distr != 0).sum()

4782

In [217]:
pred_distr_id = {}
pred_distr_token = {}
for i, id in enumerate(encoded_static_distr.argmax(axis=1)):
    pred_distr_id[i] = id
    pred_distr_token[encoded_id2token[i]] = encoded_id2token[id]

In [221]:
pred_distr_id

{0: 139,
 1: 143,
 2: 5,
 3: 146,
 4: 65,
 5: 128,
 6: 70,
 7: 24,
 8: 67,
 9: 117,
 10: 89,
 11: 17,
 12: 45,
 13: 154,
 14: 20,
 15: 199,
 16: 88,
 17: 31,
 18: 68,
 19: 53,
 20: 84,
 21: 108,
 22: 33,
 23: 3,
 24: 27,
 25: 93,
 26: 21,
 27: 29,
 28: 90,
 29: 7,
 30: 23,
 31: 307,
 32: 125,
 33: 389,
 34: 71,
 35: 183,
 36: 102,
 37: 125,
 38: 68,
 39: 103,
 40: 232,
 41: 109,
 42: 84,
 43: 9,
 44: 17,
 45: 169,
 46: 518,
 47: 60,
 48: 159,
 49: 86,
 50: 4,
 51: 158,
 52: 102,
 53: 0,
 54: 3,
 55: 220,
 56: 392,
 57: 109,
 58: 0,
 59: 70,
 60: 9,
 61: 23,
 62: 0,
 63: 37,
 64: 75,
 65: 83,
 66: 277,
 67: 79,
 68: 229,
 69: 11,
 70: 4,
 71: 85,
 72: 124,
 73: 144,
 74: 271,
 75: 10,
 76: 126,
 77: 436,
 78: 59,
 79: 197,
 80: 405,
 81: 253,
 82: 22,
 83: 546,
 84: 31,
 85: 87,
 86: 337,
 87: 28,
 88: 36,
 89: 120,
 90: 3,
 91: 4,
 92: 46,
 93: 94,
 94: 2,
 95: 21,
 96: 90,
 97: 432,
 98: 224,
 99: 40,
 100: 342,
 101: 196,
 102: 248,
 103: 7,
 104: 62,
 105: 49,
 106: 101,
 107: 306,
